In [1]:
import numpy as np
import librosa
from scipy.io import wavfile

In [2]:
# Set tempo (120 BPM / 4 = 30 cycles per minute = 0.5 cycles per second)
bpm = 120 / 4
cycle_duration = 60 / bpm  # 2 seconds per cycle
sr = 44100  # Sample rate

def generate_kick(duration, sr=44100):
    """Generate a simple kick drum sound"""
    t = np.linspace(0, duration, int(sr * duration))
    # Pitch envelope: starts at 150Hz, drops to 50Hz
    freq = 150 * np.exp(-10 * t) + 50
    # Amplitude envelope
    amp = np.exp(-5 * t)
    kick = amp * np.sin(2 * np.pi * freq * t)
    return kick

def generate_supersaw(freq, duration, sr=44100, detune=0.1):
    """Generate a supersaw waveform (multiple detuned sawtooth waves)"""
    t = np.linspace(0, duration, int(sr * duration))
    saw = np.zeros_like(t)
    
    # Create 7 detuned sawtooth waves
    for i in range(-3, 4):
        detune_factor = 1 + (i * detune / 3)
        saw += librosa.tone(freq * detune_factor, sr=sr, length=len(t))
    
    # Normalize and apply envelope
    saw = saw / 7
    envelope = np.exp(-2 * t)  # Simple decay envelope
    return saw * envelope

def apply_ducking(signal, kick_pattern, attack=0.1, sr=44100):
    """Apply sidechain ducking effect"""
    ducking_env = np.ones_like(signal)
    attack_samples = int(attack * sr)
    
    for kick_time in kick_pattern:
        start_idx = int(kick_time * sr)
        end_idx = min(start_idx + attack_samples, len(signal))
        if start_idx < len(signal):
            duck_curve = np.linspace(0.3, 1.0, end_idx - start_idx)
            ducking_env[start_idx:end_idx] = np.minimum(
                ducking_env[start_idx:end_idx], 
                duck_curve
            )
    
    return signal * ducking_env

def note_to_freq(note_name, octave):
    """Convert note name and octave to frequency"""
    notes = {'c': 0, 'd': 2, 'e': 4, 'f': 5, 'g': 7, 'a': 9, 'b': 11}
    semitone = notes[note_name.lower()]
    # A4 = 440 Hz, calculate relative to that
    midi_note = 12 * octave + semitone
    return 440 * (2 ** ((midi_note - 57) / 12))

# Create pattern sequences
num_cycles = 4
total_duration = num_cycles * cycle_duration
total_samples = int(total_duration * sr)

# Initialize output arrays
kick_track = np.zeros(total_samples)
bass_track = np.zeros(total_samples)
juice_track = np.zeros(total_samples)

# Kick pattern: 4 kicks per cycle (bd!4:8 means 4 kicks per cycle)
kick_times = []
for cycle in range(num_cycles):
    for i in range(4):
        kick_time = cycle * cycle_duration + i * (cycle_duration / 4)
        kick_times.append(kick_time)
        start_idx = int(kick_time * sr)
        kick_sound = generate_kick(0.3, sr)
        end_idx = min(start_idx + len(kick_sound), total_samples)
        kick_track[start_idx:end_idx] += kick_sound[:end_idx - start_idx]

# Bass pattern: a1 a2 a2 a2 a1 a2 c3 a2, each repeated 16 times per cycle
bass_notes = ['a1', 'a2', 'a2', 'a2', 'a1', 'a2', 'c3', 'a2']
bass_pattern = []
for note in bass_notes:
    bass_pattern.extend([note] * 16)

note_duration = cycle_duration / len(bass_pattern)

for cycle in range(num_cycles):
    for i, note in enumerate(bass_pattern):
        note_name = note[0]
        octave = int(note[1])
        freq = note_to_freq(note_name, octave)
        
        start_time = cycle * cycle_duration + i * note_duration
        start_idx = int(start_time * sr)
        
        note_sound = generate_supersaw(freq, note_duration, sr)
        end_idx = min(start_idx + len(note_sound), total_samples)
        bass_track[start_idx:end_idx] += note_sound[:end_idx - start_idx] * 0.4

# Juice pattern: a f d d, each repeated 2 times (total 8 notes per cycle)
juice_notes = ['a', 'f', 'd', 'd']
juice_pattern = []
for note in juice_notes:
    juice_pattern.extend([note] * 2)

juice_note_duration = cycle_duration / len(juice_pattern)

for cycle in range(num_cycles):
    for i, note in enumerate(juice_pattern):
        freq = note_to_freq(note, 3)  # Default to octave 3
        
        start_time = cycle * cycle_duration + i * juice_note_duration
        start_idx = int(start_time * sr)
        
        note_sound = generate_supersaw(freq, juice_note_duration, sr, detune=0.15)
        end_idx = min(start_idx + len(note_sound), total_samples)
        juice_track[start_idx:end_idx] += note_sound[:end_idx - start_idx] * 0.8

# Apply ducking to bass and juice tracks
bass_track = apply_ducking(bass_track, kick_times, attack=0.1, sr=sr)
juice_track = apply_ducking(juice_track, kick_times, attack=0.1, sr=sr)

# Mix all tracks
master = kick_track + bass_track + juice_track

# Normalize to prevent clipping
master = master / np.max(np.abs(master)) * 0.9

# Convert to 16-bit PCM
master_int = np.int16(master * 32767)

In [3]:
# Save to WAV file
wavfile.write('strudel_output.wav', sr, master_int)

print(f"Generated {total_duration:.2f} seconds of audio")
print(f"Saved to 'strudel_output.wav'")
print(f"Tempo: {bpm} BPM (120/4)")
print(f"Cycle duration: {cycle_duration} seconds")

Generated 8.00 seconds of audio
Saved to 'strudel_output.wav'
Tempo: 30.0 BPM (120/4)
Cycle duration: 2.0 seconds
